# **MODELOS DE SÉRIES TEMPORAIS PARA PREDIÇÃO PLUVIOMÉTRICA**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 4

## **Tratamento da base de dados**

Importando a base de dados relativa a quantidade de chuva mensal na cidade de São Paulo

In [2]:
chuva = pd.read_csv('Chuva_Mensal.csv',sep=';')

Boas Práticas para análise de dados:

*  Analisar as variaveis do dataframe importado;
*  Verificar o tamanho dataframe;
*  Verificar como o python reconhece as variáveis;
*  Verificar a existência de valores faltantes.

In [3]:
chuva.head()

Possiveis problemas : virgulas e traços (valores nao preenchidos)

Verificando o tamanho da nossa base de dados

In [4]:
chuva.shape

Análise dos tipos de atributos.
*   object: strings
*   int64: inteiros
*   float64: reais
*   complex: complexos

In [5]:
chuva.dtypes

Temos todas as variáveis como strings ou objetos

Inicialmente vamos Substituir as vírgulas por pontos

Função lambda -> função anônima (simplifica a escrita da função) - criação direta de função.

Com o .apply a função criada irá percorrer todos valores e subtituir , por .

In [6]:
chuva = chuva.apply(lambda x: x.str.replace(',','.'))
chuva.head()

In [7]:
chuva.dtypes

As variáveis ainda não são do tipo float.

Por isso iremos transformar variáveis categóricas em numéricas e substituir os traços pela média

In [8]:
print(chuva)

In [9]:
chuva['Janeiro'] =  pd.to_numeric(chuva['Janeiro'].str.replace('---', '305.79'))
chuva['Fevereiro'] =  pd.to_numeric(chuva['Fevereiro'].str.replace('---', '249.3'))
chuva['Março'] =  pd.to_numeric(chuva['Março'].str.replace('---', '227.97'))
chuva['Abril'] =  pd.to_numeric(chuva['Abril'].str.replace('---', '93.64'))
chuva['Maio'] =  pd.to_numeric(chuva['Maio'].str.replace('---', '86.67'))
chuva['Junho'] =  pd.to_numeric(chuva['Junho'].str.replace('---', '59.66'))
chuva['Julho'] =  pd.to_numeric(chuva['Julho'].str.replace('---', '54.6'))
chuva['Agosto'] =  pd.to_numeric(chuva['Agosto'].str.replace('---', '27.85'))
chuva['Setembro'] =  pd.to_numeric(chuva['Setembro'].str.replace('---', '83.8'))
chuva['Outubro'] =  pd.to_numeric(chuva['Outubro'].str.replace('---', '139.89'))
chuva['Novembro'] =  pd.to_numeric(chuva['Novembro'].str.replace('---', '145.04'))
chuva['Dezembro'] =  pd.to_numeric(chuva['Dezembro'].str.replace('---', '236.04'))
print(chuva)

In [10]:
chuva.dtypes

Excluir as duas últimas linhas - a média e o ultimo valor para realizarmos a previsão e comparar depois


In [11]:
chuva2 = chuva.drop([36,37])
print(chuva2)

Estrutura da base de dados está ideal?

Teremos que formatar a tabela para criar nossa série temporal

In [12]:
chuva2 = chuva2.drop(columns='Ano')
chuva2.head()

Temos que configurar a tabela de maneira a ordenar os registros por linha

Transformar em Array resolveria o problema?

In [13]:
chuva3 = chuva2.values
chuva3

Teriamos vários vetores por conta da quebra de linhas

Transformando em uam lista sem quebras


In [14]:
chuva4 = list(chuva3.flatten())
print(chuva4)

Transformar em lista sem quebra, por isso utilizamos o comando .flatten()

Criando um range de datas, mensalmente a partir de 1985, e a partir da quantidade de registros na lista chuva4

In [15]:
indice = pd.date_range('1985', periods = len(chuva4), freq = 'M')
indice

Criando a série temporal

In [16]:
serie = pd.Series(chuva4, index = indice)
serie

In [17]:
serie.plot()
plt.show()

Análise e preparação da Série Temporal

Analisando a tendência

Calculando a média móvel para suavização da série.

*   Utilizando janela de 6 meses
*   Retiramos a média dessa janela


In [21]:
media_movel = serie.rolling(window=12)
media_movel = media_movel.mean()

In [22]:
plt.plot(media_movel);

In [23]:
plt.plot(serie, label='Série Real')
plt.plot(media_movel,color='red', label='Média Móvel 6 meses')
plt.legend(loc='best')
plt.show()

Existe oscilação na média móvel, porém bem mais suave do que da série real.  (redução de amplitude)

A média movel facilita na verificação da existencia de tendência de crescimento ou decrescimento da série.

Podemos aumentar a janela da média móvel para verificar mais claramente a ausência do fator de tendência

**Decomposição**

Iremos realizar a decomposição da série para analisarmos a tendencia, sazonalidade e resíduos

In [24]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [25]:
decomposicao = seasonal_decompose(serie)

In [26]:
decomposicao.plot()

No primeiro gráfico temos a série temporal real.

No segundo a tendencia, que evidencia há ausência de tendencia crescente ou decrescente. Existe uma oscilação porém não é possível identificar um ciclo bem definido.

No terceiro a sazonalidade forte e constante , os picos não aumentam nem dinuem, eles se matem constantes ao longo da série, e com intervalos bem definidos

E no quarto gráfico podemos observar os resíduos que estão aparentemente aleatorios em torno de zero.

Análise de resíduos é muito importante pois ela que nos mostram a qualidade do nosso modelo, como verememos mais a frente.

### **Normalidade e Transformação**

In [27]:
import scipy.stats as stats

Verificando a normalidade dos dados da série

In [28]:
stats.probplot(serie, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Podemos verificar que os dados não se aproximam de uma distribuição normal, apenas alguns pontos estão proximos da linha de referencia.
E alguns pontos principalmente no início e fim da série estão bem distantes da linha de referência.

Fazendo o teste de Sahpiro-Wilk para confirmar a ausência de normalidade

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [29]:
e, p = stats.shapiro(serie)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Vamos tentar aproximar de uma distribuição normal

Inicialmente iremos realizar uma tranformação Transformação por log, buscando diminuir variância e melhorar normalidade.

In [30]:
serie2 = np.log(serie)
serie2

In [31]:
e, p = stats.shapiro(serie2)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Como na função logarítimica não podemos ter zero ou valores negativos por isso a função apresenta erro.

Então tentaremos outro tipo de transformação.

Transformação por raiz cúbica é muito utilizada quando na série temos dados com valor zero ou negativos.

In [32]:
serie3 = np.sign(serie)*abs(serie)**(1/3)
serie3

In [33]:
stats.probplot(serie3, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Observamos uma melhora significativa, em relação ao ajuste dos dados, porém ainda temos muitos pontos distantes da linha de referencia, o que possivelmente resultará em uma série que ainda não é aproximadamente normal.

Fazendo o teste de hipótese para verificar se os dados seguem uma distribuição normal.

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [34]:
e, p =stats.shapiro(serie3)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Ainda não temos uma série normal.

In [35]:
import seaborn as sns
sns.distplot(serie);

In [36]:
sns.distplot(serie3);

Observamos que a série melhora bastante sua disposição, e se aproxima mais da normalidade.

Sempre temos que buscar uma série que aproxima de uma distribuição normal para termos modelos melhores.

Não é recomendado ficar tentando transformações sobrepostas para obter a normalidade.

### **Estacionaridade**

Média e variância constantes independente do tempo que a série é observada.

In [37]:
import statsmodels.tsa.stattools

Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)

Ha = não é estacionário: estatística do teste > valor crítico

Ho = é estacionário:  estatística do teste < valor crítico

In [38]:
kpss = statsmodels.tsa.stattools.kpss(serie3)
print('Estatítica do teste: {:.4f}'.format(kpss[0]))
print('p_valor: {:.4f}'.format(kpss[1]))
print('Valores Críticos:')
for chave, valor in kpss[3].items():
   print('{}: {:.4f}'.format(chave, valor))

Caso não fosse estacionária poderia tentar fazer a diferenciação:

serie5 = np.diff(serie3)

### **Autocorrelação**

In [39]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [40]:
plot_acf(serie3, lags=20)
plt.show()

ACF - Autocorrelação de valores vizinhos

O gráfico indica que temos autocorrelação, entre os valores da própria série, e ainda mostra uma forte sazonalidade.

In [41]:
plot_pacf(serie, lags=20)
plt.show()

PACF - correlação entre valores intercalados - autocorrelação Parcial

Temos muitos valores que ultrapassam o intvalo, logo os dados são autocorrelacionados quando analisamos os valores intercalados.


A série é autocorrelacionada, mas não tem problema, pois esse é um pressuposto da análise de modelos de séries temporais.

Logo temos uma série autocorrelacionada, e estacionária, porem seus dados não são normais

## **Modelo AR**

Modelo AR - Modelo Autoregressivo

Modelo ARIMA:(p,d,q)

*   p: ordem da autoregressão
*   d: grau de diferenciação (para transformar em estacionario)
*   q: Ordem de média móvel dos reísduos

Obs: cuidado para nao confundir a Média móvel que iremos analisar não é a média móvel que construímos para suavizar a tendência.
Essa média móvel está relacionada a combinação linear dos erros de valores passados

Modelo ar: (p,0,0)

Só colocamos valores numéricos para a ordem de autoregressão

In [42]:
from statsmodels.tsa.arima.model import ARIMA

Criando o Modelo AR

In [43]:
modelo_ar = ARIMA(serie3, order = (1,0,0))

Temos que analisar o modelo

In [44]:
resultado = modelo_ar.fit()
print(resultado.summary())

O AIC é a métrica mais recomendada para avaliar o modelo uma vez que existem alguns modelos que não retornam o BIC.

Obs.: O recomendado é termos valores de AR até 10, pois depois que incluirmos a sazonalidade o processo pode ficar muito demorado, além do tempo computacional aumentar, o modelo geralmente não melhora significativamente após esse valor.

Para identificar o valor que devemos incluir na métrica AR, devemos analisar o gráfico de acf

In [45]:
plot_acf(serie3, lags=20)
plt.show()

É interessante analisar os lags que tiveram mudanças bruscas, neste caso o 3 , pois não é correlacionado com o lag 2, o 4 pois "virou" autocorrelacionado, o 9 e 10 pelos mesmo motivos.

In [46]:
modelo_ar3 = ARIMA(serie3, order = (3,0,0))
resultado3 = modelo_ar3.fit()
modelo_ar4 = ARIMA(serie3, order = (4,0,0))
resultado4 = modelo_ar4.fit()
modelo_ar9 = ARIMA(serie3, order = (9,0,0))
resultado9 = modelo_ar9.fit()
modelo_ar10 = ARIMA(serie3, order = (10,0,0))
resultado10 = modelo_ar10.fit()

resultado3.aic

In [47]:
resultado4.aic

In [48]:
resultado9.aic

In [49]:
resultado10.aic

Modelo Final

In [50]:
modelo_ar = ARIMA(serie3, order = (9,0,0))
resultado = modelo_ar.fit()

Melhor AIC: 1337.922 = (9,0,0)

### **Análise dos Resíduos**

In [51]:
residuos = resultado.resid
residuos

In [52]:
residuos.plot()
plt.show()

Analise no gráfico de resíduos -> média e a variância.

O ideal é que a média seja aproximadamente zero e a variância constante. (ruído branco)

Observa-se que os resíduos tem média zero ao longo da serie.

Já a variância não aparenta ser constante, pois sua amplitude varia.

**Normalidade**

Testando a normalidade dos resíduos

In [53]:
stats.probplot(residuos, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Aparentemente não teremos dados normais, observa-se um distanciamento dos pontos em relação a linha de referência no início e fim do grafico.

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [54]:
e, p = stats.shapiro(residuos)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Pelo teste de hipótese os dados não são normais

In [55]:
import seaborn as sns
sns.distplot(residuos);

A distribuição se aproxima de uma distribuição normal.

Pela analise de resíduos nosso modelo não ficou "perfeito" e pode apresentar falhas na previsão.

Resultados obtidos até agora em nosso modelo:


*   Distribuição dos resíduos apresenta uma média constante, sem presença de tendencia
*   Dados não normais

Agora vamos analisar a autocorrelação. Neste caso o ideal é que o resíduos NÃO SEJAM autocorrelacionados.



**Autocorrelação**

In [56]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [57]:
plot_acf(residuos, lags=20)
plt.show()

A grande maioria dos pontos esta dentro do intervalo de confiança, logo os residuos não são autocorrelacionados.

Validando a ausencia da autocorrelação com o pacf

In [58]:
plot_pacf(residuos, lags=20)
plt.show()

A grande maioria dos pontos esta dentro do intervalo de confiança, logo os residuos não são autocorrelacionadas

In [59]:
plt.plot(serie3,color='blue', label='Série Real')
plt.plot(serie3-residuos,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Os residuos conseguem seguir bem o padrão de oscilação da serie real

**Previsão**

In [60]:
resultado.fittedvalues

fittedvalues -> mostra os valores que foram ajustados com o modelo

Temos duas formas de fazer previsão: Predict e forecast

O tamanho da nossa série (Length) é 432 , porem o python começa a contabilizar no 0.

In [61]:
previsao = resultado.predict(431, end=443)
previsao

In [62]:
previsao2 = resultado.forecast(12)
previsao2

In [67]:
plt.plot(serie3,color='yellow', label='Série Real')
plt.plot(previsao,color='blue', label='Previsão')
plt.plot(serie3-residuos,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Como os dados foram transformados, a escala apresenta no grafico nao é a escala real da série.

Como tinhamos retirado a raiz cúbica, iremos elevar a 3 agora.

In [64]:
prev_escala = pd.DataFrame(previsao ** 3)
prev_escala

Plotando os valores reais

In [65]:
pd.concat([serie,prev_escala]).plot();

## **Modelo MA**

Modelo de Média Móvel - Está relacionado ao erro de regressão. Ele faz uma relação linear dos erros anteriores, erros passados.

Esta média móvel é diferente da realizada anteriormente para analisar as tendencias.

Modelo arima:(p,d,q)



*   p: ordem da autoregressão
*   d: grau de diferenciação (para transformar em estacionario)
*   q: Ordem de média móvel dos resíduos


Modelo MA: (0,0,q)

Existe uma limitação sugerida de chegarmos até aproximadamente 10 na escolha da ordem da média movel

In [68]:
from statsmodels.tsa.arima.model import ARIMA

Criando o modelo MA

In [69]:
modelo_ma1 = ARIMA(serie3, order = (0,0,1))
resultado1 = modelo_ma1.fit()
modelo_ma2 = ARIMA(serie3, order = (0,0,2))
resultado2 = modelo_ma2.fit()
modelo_ma3 = ARIMA(serie3, order = (0,0,3))
resultado3 = modelo_ma3.fit()
modelo_ma4 = ARIMA(serie3, order = (0,0,4))
resultado5 = modelo_ma4.fit()
modelo_ma5 = ARIMA(serie3, order = (0,0,5))
resultado5 = modelo_ma5.fit()
modelo_ma6 = ARIMA(serie3, order = (0,0,6))
resultado6 = modelo_ma6.fit()
modelo_ma7 = ARIMA(serie3, order = (0,0,7))
resultado7 = modelo_ma7.fit()
modelo_ma8 = ARIMA(serie3, order = (0,0,8))
resultado8 = modelo_ma8.fit()
modelo_ma9 = ARIMA(serie3, order = (0,0,9))
resultado9 = modelo_ma9.fit()
modelo_ma10 = ARIMA(serie3, order = (0,0,10))
resultado10 = modelo_ma10.fit()
resultado1.aic

In [70]:
resultado2.aic

In [71]:
resultado3.aic

In [72]:
resultado4.aic

In [73]:
resultado5.aic

In [74]:
resultado6.aic

In [75]:
resultado7.aic

In [76]:
resultado8.aic

In [77]:
resultado9.aic

In [78]:
resultado10.aic

Para o modelo de MA, podemos testar os 10 valores para selecionar o modelo que tem o menor AIC.

In [79]:
resultado_ma = modelo_ma8.fit()
print(resultado_ma.summary())

Modelo AR: AIC = 1337.922 (9,0,0)

Modelo MA: AIC = 1411.696 (0,0,8)

### **Análise dos Resíduos**

Fazendo a analise de resíduos para o modelo que tem o melhor AIC.

Lembrando que o ideal é que os resíduos tenha media zero e variancia contante ao longo da série. (Ruído Branco)

In [80]:
residuos_ma = resultado_ma.resid

In [81]:
residuos_ma.plot()
plt.show()

Existem muitos picos, principalmente na parte inferiror, mas no geral temos uma média aparentemente constate em torno de zero.

**Normalidade**

Analisando a Normalidade dos resíduos

In [82]:
stats.probplot(residuos_ma, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Observamos que os pontos iniciais se distanciam da reta de referencia. Aparentemente não teremos uma distribuição normal.

Realizando o teste de hipótese de Shapiro Wilk

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [83]:
e, p = stats.shapiro(residuos_ma)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Temos que p-valor é < que 0,05 por tanto nossa distribuição não é normal

In [84]:
import seaborn as sns
sns.distplot(residuos_ma);

No histograma observamos uma assimetria e alguns picos que podems estar atrapalhando a normalidade dos resíduos

**Autocorrelação**

Verificando a autocorrelação dos resíduos

In [85]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [86]:
plot_acf(residuos_ma, lags=30)
plt.show()

Podemos verificar que existem muitos pontos fora do intervalo de confiança, apontando que existe uma autocorrelação.

In [87]:
plot_pacf(residuos_ma, lags=30)
plt.show()

O mesmo observamos na autocorrelação parcial, onde temos muitos pontos fora do intervalo de confiança, apontando que axite uma autocorrelação.

Para o modelo MA temos que os resíduos:

*   Variância não constante
*   Não são normais
*   Estão autocorrelacionados

In [88]:
plt.plot(serie3, label='Série Real')
plt.plot(serie3-residuos_ma,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Observa-se uma coincidencia da foram em que os resíduos está oscilando versus o real.

**Previsão**

In [89]:
resultado_ma.fittedvalues

Mostra todos os valores ajustados pelo modelo

Previsão para 12 meses,  lembrando que o python inicia em 0, por isso temos que diminuir 1 em relaçã ao tamanho da série.

In [92]:
previsao_ma = resultado_ma.predict(start=431, end=443)
previsao_ma

In [93]:
plt.plot(serie3,color='yellow', label='Série Real')
plt.plot(previsao_ma,color='blue', label='Previsão')
plt.plot(serie3-residuos_ma,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Aparentemente a predição não fica boa, pouca amplitude dos dados. E conforme analisado nos resíduos não temos um modelo adequado para predizer os dados utilizando o Modelo de média Móvel (MA).

Retornando a Escala original da série.

Neste caso como retiramos a raiz vubica devemos elevar ao cubo a nossa série.

In [94]:
prev_escala_ma = pd.DataFrame(previsao_ma**3)
prev_escala_ma

Concatenando a série original com a previsão encontrada.

In [95]:
pd.concat([serie,prev_escala_ma]).plot();

## **Modelo ARMA**

Modelo Autoregressivo + Média Móvel

No modelo ARMA não incluímos nenhum parâmetro apenas para a diferenciação.

Modelo arima:(p,d,q)

*   p: ordem da autoregressão
*   d: grau de diferenciação (para transformar em estacionario)
*   q: Ordem de média móvel dos reísduos

Modelo arma: (p,0,q)

Modelo AR: AIC = 1337.922 (9,0,0)

Modelo MA: AIC = 1411.696 (0,0,8)

Nem sempre a combinação dos resultados encontrados anterioemnte será linear apra acharmos o modelo que minimiza o AIC no ARMA. Temos que verificar tambem se  tempo computacional não ficará muito elevado.

In [96]:
from statsmodels.tsa.arima.model import ARIMA

In [111]:
modelo_arma = ARIMA(serie3, order = (10,0,3))

In [112]:
resultado_arma = modelo_arma.fit()
print(resultado_arma.summary())

Observe que ele considerou o Modelo ARMA, ja desconsiderou o I de ARIMA. Uma vez que não incluimos e não se fez necessário valores para diferenciação.

Utilizando a combinação ARMA(9,0,8) teremos problemas de inversibilidade, problema da algébra linear onde os coeficiente MA não são invertíveis.

PAra evistar esse problema ao combinar modelo AR e MA devemos utilizar valores mais baixos para a ordem da média movel.


Modelo AR: AIC = 1337.922 (9,0,0)

Modelo MA: AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

### **Análise dos Resíduos**

Analisando os resíduos do modelo criado

In [113]:
residuos_arma = resultado_arma.resid

In [114]:
residuos_arma.plot()
plt.show()

A aprentemente a média é constante em torno de zero, porem existe alguns picos.

**Normalidade**

In [115]:
stats.probplot(residuos_arma, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

A normalidade dos residuos comparada a graficos e modelos anteriores graficamete aprece mais proxim da linah de referência.

Testando a normalidade pelo teste estatístico de Shapiro-Wilk

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [116]:
e, p = stats.shapiro(residuos_arma)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Ainda não temos uma distribuição normal.

In [117]:
import seaborn as sns
sns.distplot(residuos_arma);

O histograma mostra uma leve assimetria dos dados. Porem visualmente essa distribuição está muito proxima da normalidade. Não tendo modelos melhores, poderíamos aceitar o ARMA.

**Autocorrelação**

Verificando se existe autocorrelação entre os resíduos.

In [118]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [119]:
plot_acf(residuos_arma, lags=30)
plt.show()

Os resíduos não são autocorrelacionados, por apresentarem a amioria dos lags dentro do intervalo de confiança

In [120]:
plot_pacf(residuos_arma, lags=30)
plt.show()

Verificando pelo grafico de correlação aprcial temos que os resíduos não são autocorrelacionados, por apresentarem a amioria dos lags dentro do intervalo de confiança

In [121]:
plt.plot(serie3, label='Série Real')
plt.plot(serie3-residuos_arma,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Observe que nesse modelo a amplitude se ajustou melhor, e está bem proximo a série original.
Bem uniforme e proxima a série original.

**Previsão**

In [122]:
resultado_arma.fittedvalues

Valores ajustados encontrado pelo modelo.

Realizando a previsão para 12 meses.

In [123]:
previsao_arma = resultado_arma.predict(start=431, end=455)
previsao_arma

Plotando o gráfico para previsão

In [124]:
plt.plot(serie3,color='red', label='Série Real')
plt.plot(previsao_arma,color='blue', label='Previsão')
plt.plot(serie3-residuos_arma,color='yellow', label='Resíduos')
plt.legend(loc='best')
plt.show()

Revertendo a escala para voltar a original

In [125]:
prev_escala_arma = pd.DataFrame(previsao_arma ** 3)
prev_escala_arma

In [126]:
pd.concat([serie,prev_escala_arma]).plot();

## **Modelo ARIMA**

Modelo ARIMA:(p,d,q)

Neste modelo utilizamos os três parâmetros

*   p: ordem da autoregressão
*   d: grau de diferenciação (para transformar em estacionario)
*   q: Ordem de média móvel dos reísduos


Modelo AR: AIC = 1337.922 (9,0,0)

Modelo MA: Melhor AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

Para termos um modelo de ARIMA iremos colocar um valor para diferenciação mas na prática, não foi necessário realizar diferenciação, pois a serie ja era estacionária.

In [127]:
from statsmodels.tsa.arima.model import ARIMA

Vamos manter a configuração do ARMA e adicionar a diferenciação

Criando o modelo do ARIMA

In [128]:
modelo_arima = ARIMA(serie3, order = (10,1,2))

Verificando a saída do resumo do modelo

In [129]:
resultado_arima = modelo_arima.fit()
print(resultado_arima.summary())

Modelo AR: Melhor AIC = 1339.919 (10,0,0)

Modelo MA: Melhor AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

Modelo ARIMA: Melhor AIC = 1319.230 (10,1,2)

### **Análise dos Resíduos**

Validar o modelo ARIMA - a partir da análise de resíduos

In [130]:
residuos_arima = resultado_arima.resid

In [131]:
residuos_arima.plot()
plt.show()

Verificar se os residuos apresentam tendencia , se tem média contante em torno de zero e se a amplitude não está discrepante.

O ideal é que o resíduo seja um reído branco com media zero e variancia constante.



**Normalidade**

Analisando a normalidade dos resíduos

In [132]:
stats.probplot(residuos_arima, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

O Qq-plot, aparentemente não está ruim. no começo e no final da série apresentam dados um pouco dispersos.

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

A partir do teste numérico não temos normalidade nos resíduos.

In [133]:
e, p = stats.shapiro(residuos_arima)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

In [134]:
import seaborn as sns
sns.distplot(residuos_arima);

Na análise do histograma, para verificar a disposição da série observamos que sua distribuição aproxima de uma normal, visualmente

**Autocorrelação**

Verificando a autocorrelação residual.

In [135]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [136]:
plot_acf(residuos_arima, lags=30)
plt.show()

A maioria dos lags estão dentro do intervalo de confiança, logo podemos dizer que pelo gráfico de autocorrelação os resíduals não são autocorrelacionados.

In [137]:
plot_pacf(residuos_arima, lags=30)
plt.show()

A partir da analise da autocorrelação parcial temos que a maioria dos lags estão dentro do intervalo de confiança, logo podemos que os resíduals não são autocorrelacionados.

In [138]:
plt.plot(serie3, label='Série Real')
plt.plot(serie3-residuos_arima,color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

Plotando o grafico de resíduos e da série observamos que os resíduos estão adequados, dentro do que esperávamos. Ele consegue seguir as oscilações da série.

Observe que fazer a diferenciação ou não neste caso não irá alterar muito o modelo, pois ja temos uma série estacionária.

**Previsão**

Verificando os valores ajustados.

A série apresenta valores negativos pois realizamos a trasnformação e a diferenciação.

In [139]:
resultado_arima.fittedvalues

In [140]:
previsao_arima = resultado_arima.predict(start=430, end=443)
previsao_arima

Temos que fazer uma transformação de dados - utilizando o predict isso daria mais trabalho pois não realiza a inversão da diferenciação de maneira automática.

Fazendo a previsão utilizando o método forecast -> ja resolve o problema da diferenciação

In [141]:
previsao_arima2 = resultado_arima.forecast(12)
previsao_arima2

Revertendo a raiz cúbica

In [142]:
prev_escala_arima = pd.DataFrame(previsao_arima2**3)
prev_escala_arima

Observe que não temos as datas com essa função.

Concatenando a série original com a previsão do arima

In [143]:
pd.concat([serie,prev_escala_arima]).plot();

Aparentemente obtemos uma boa previsão, com uma boa amplitude.

## **Modelo AUTO ARIMA**

Realiza todas as comparações de modelo e retorna o melhor modelo.

A função autoarima inclui a sazonalidade.

Na nossa série aparentemente temos uma sazonalidade bem definida.

Modelo arima:(p,d,q)

*   p: ordem da autoregressão
*   d: grau de diferenciação (para transformar em estacionario)
*   q: Ordem de média móvel dos reísduos


Modelo SARIMA: (p,d,q)(P,D,Q)

Modelo AR: Melhor AIC = 1337.922 = (9,0,0)

Modelo MA: Melhor AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

Modelo ARIMA: Melhor AIC = 1319.230 (10,1,2)

In [ ]:
#!pip install pmdarima

In [144]:
from pmdarima.arima import auto_arima

Criando modelo utilizando o autoarima.

Parâmetros:
*   Trace: Apresenta a lista de modelos testados
*   Stepwise: Seleção gradual do modelo "ideal", é um processo mais rápido porém menos minuncioso.
*   Seasonal: analisar a sazonalidade ou não
*   Parametros máximos a serem analidados SARIMA (p,d,q)(P,D,Q) -> interessante dobrar o default
*   M: período sazonal, no nosso caso é mensal.

In [145]:
modelo_auto = auto_arima(serie3, trace = True, stepwise = False, seasonal=True, max_p=10, max_q=10,
                          max_P=4, max_Q=4, start_p=0, start_q=0, start_P=0, start_Q=0, m=12,)

Encontramos o modelo com melhor AIC facilmente utilizando:

In [146]:
print(modelo_auto.aic())

Modelo AR: Melhor AIC = 1339.919 (10,0,0)

Modelo MA: Melhor AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

Modelo ARIMA: Melhor AIC = 1319.230 (10,1,2)

Modelo_AUTOARIMA: AIC=1270.5894 ARIMA(0,0,0)(1,0,3)

Fazendo o ajuste utilizando o modelo autoarima

In [147]:
resultado_auto = modelo_auto.fit(serie3)
print(resultado_auto.summary())

O reusltado do autoarima é o melhor modelo pelo AIC.

### **Análise dos Resíduos**

Realizando a análise de residuos do modelo gerado pelo autoarima.

Observe que a função de analise de residuos é diferente pois estamos utilizando outra biblioteca.

In [148]:
residuos_auto = resultado_auto.resid
residuos_auto()

Plotando os residuos

Aparentemente temos uma média constante (não há tendencia)
Variância não se amntém contante, amplitude varia




In [149]:
plt.plot(residuos_auto())
plt.show()

**Normalidade**

In [150]:
stats.probplot(residuos_auto(), dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Realizando o teste de normalidade para os residuos, observamos que os dados iniciais e finais se afastam um pouco da linah de referência.


**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [151]:
e, p = stats.shapiro(residuos_auto())
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

O p-valor é < que 0,05 indicando que a distribuição não é normal

In [152]:
import seaborn as sns
sns.distplot(residuos_auto());

Temos um pico na distribuição, porem os dados se aproximam de uma normalidade

**Autocorrelação**

In [153]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [154]:
plot_acf(residuos_auto(), lags=30)
plt.show()

Existem lags que saem  do intervalo de confiaça, porem a maioria fica dentro do intervalo. logo não há autocorrelação nos resíduos

In [155]:
plot_pacf(residuos_auto(), lags=30)
plt.show()

Para analise de correlação parcial temos o mesmo, existem lags que saem  do intervalo de confiaça, porem a maioria fica dentro do intervalo. logo não há autocorrelação nos resíduos

In [156]:
plt.plot(serie3, label='Série Real')
plt.plot(serie3-residuos_auto(),color='red', label='Resíduos')
plt.legend(loc='best')
plt.show()

plotando a sére com os resíduos, observamo que em geral seguem bem os dados , porem no início temos uma pequena diferença.

**Previsão**

Fazendo a previsão de 12 meses.

In [157]:
previsao_auto = resultado_auto.predict(n_periods=12)
previsao_auto

Transformando os dados , retornando a escala inicial -> elevar ao cubo

In [158]:
prev_escala_auto = pd.DataFrame(previsao_auto ** 3, columns=['Previsão_SARIMA'])
prev_escala_auto

In [159]:
pd.concat([serie,prev_escala_auto]).plot();

A previsão consegue seguir o padrão da série.

## **Análise final do projeto**

Para comparar o modelo utilizamos a métrica AIC

**COMPARAÇÃO DOS MODELOS**

Modelo AR: Melhor AIC = 1339.919 (10,0,0)

Modelo MA: Melhor AIC = 1411.696 (0,0,8)

Modelo ARMA: Melhor AIC = 1273.766 (10,0,2)

Modelo ARIMA: Melhor AIC = 1319.230 (10,1,2)

**Modelo_SARIMA (AutoArima): AIC=1270.5894 ARIMA(0,0,0)(1,0,3)**

**DESEMPENHO DOS MODELOS**

Para verificar o desempenho dos modelos vamso utilizar:


*   ERRO MÉDIO ABSOLUTO (MAE)
*   RAIZ DO ERRO QUADRÁTICO MÉDIO (RMSE)



Vamos realizar a comparação utilizando os meses de 2021, que foram separados inicialmente.

Criando uma lista apra os valores reais

In [ ]:
lista = [373.3, 174.1, 137.8, 55.7]
valores_reais = pd.DataFrame(lista, columns = ['valores reais'])
print(valores_reais)

Os 4 primeiros valores previstos de cada modelo, observe que alguns colocamos para iniciar em dezembro e outros em janeiro.

In [ ]:
auto = prev_escala_auto.iloc[0:4]
ar = prev_escala.iloc[1:5]
ma = prev_escala_ma.iloc[1:5]
arima = prev_escala_arima.iloc[0:4]
arma = prev_escala_arma.iloc[1:5]


In [ ]:
auto

In [ ]:
ar

Excluindo índice para os que estavam com datas

In [ ]:
pd.DataFrame.reset_index(ar, drop=True, inplace=True)
pd.DataFrame.reset_index(ma, drop=True, inplace=True)
pd.DataFrame.reset_index(arma, drop=True, inplace=True)
pd.DataFrame.reset_index(auto, drop=True, inplace=True)
pd.DataFrame.reset_index(arima, drop=True, inplace=True)

Criando um dataframe para concatenar os valores reais e a predição do autoarima

In [ ]:
desempenho = pd.concat([valores_reais, auto, ar, ma, arma, arima],axis=1)
desempenho

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

**ERRO MÉDIO ABSOLUTO (MAE)**

In [ ]:
print('ERRO MÉDIO ABSOLUTO (MAE)')
mae_sarima = mean_absolute_error(desempenho['valores reais'], desempenho.iloc[:, 1])
print('SARIMA: {:.2f}'.format(mae_sarima))
mae_ar = mean_absolute_error(desempenho['valores reais'], desempenho.iloc[:, 2])
print('AR: {:.2f}'.format(mae_ar))
mae_ma = mean_absolute_error(desempenho['valores reais'], desempenho.iloc[:, 3])
print('MA: {:.2f}'.format(mae_ma))
mae_arma = mean_absolute_error(desempenho['valores reais'], desempenho.iloc[:, 4])
print('ARMA: {:.2f}'.format(mae_arma))
mae_arima = mean_absolute_error(desempenho['valores reais'], desempenho.iloc[:, 5])
print('ARIMA: {:.2f}'.format(mae_arima))

**ERRO QUADRÁTICO MÉDIO (MSE)**

In [ ]:
print('ERRO QUADRÁTICO MÉDIO (MSE)')
mse_sarima = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 1])
print('SARIMA: {:.2f}'.format(mse_sarima))
mse_ar = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 2])
print('AR: {:.2f}'.format(mse_ar))
mse_ma = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 3])
print('MA: {:.2f}'.format(mse_ma))
mse_arma = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 4])
print('ARMA: {:.2f}'.format(mse_arma))
mse_arima = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 5])
print('ARIMA: {:.2f}'.format(mse_arima))

**RAIZ DO ERRO QUADRÁTICO MÉDIO (RMSE)**

In [ ]:
print('RAIZ DO ERRO QUADRÁTICO MÉDIO (RMSE)')
rmse_sarima = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 1], squared=False)
print('SARIMA: {:.2f}'.format(rmse_sarima))
rmse_ar = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 2], squared=False)
print('AR: {:.2f}'.format(rmse_ar))
rmse_ma = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 3], squared=False)
print('MA: {:.2f}'.format(rmse_ma))
rmse_arma = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 4], squared=False)
print('ARMA: {:.2f}'.format(rmse_arma))
rmse_arima = mean_squared_error(desempenho['valores reais'], desempenho.iloc[:, 5], squared=False)
print('ARIMA: {:.2f}'.format(rmse_arima))